In [1]:
import pandas as pd
import h2o

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [3]:
import multiprocessing
print("CPU: ",multiprocessing.cpu_count())

import psutil
print("Memory: ",psutil.virtual_memory())

CPU:  72
Memory:  svmem(total=1622764929024, available=1460412657664, percent=10.0, used=156969132032, free=594238025728, active=872254349312, inactive=111586131968, buffers=0, cached=871557771264, shared=2502144000, slab=15479771136)


In [4]:
#h2o.init()
h2o.init(ip="127.0.0.1", max_mem_size_GB = 100, nthreads = 5)

Checking whether there is an H2O instance running at http://127.0.0.1:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_212"; OpenJDK Runtime Environment (build 1.8.0_212-b04); OpenJDK 64-Bit Server VM (build 25.212-b04, mixed mode)
  Starting server from /workspaces/sefik/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp98fw8q5b
  JVM stdout: /tmp/tmp98fw8q5b/h2o_sefik_started_from_python.out
  JVM stderr: /tmp/tmp98fw8q5b/h2o_sefik_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Istanbul
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,25 days
H2O cluster name:,H2O_from_python_sefik_ugsf64
H2O cluster total nodes:,1
H2O cluster free memory:,88.9 Gb
H2O cluster total cores:,72
H2O cluster allowed cores:,5
H2O cluster status:,"accepting new members, healthy"


In [5]:
hf_positive = h2o.import_file('dataset/train_true_positive_features.csv')
hf_negative = h2o.import_file('dataset/train_true_negative_features.csv')
hf = hf_positive.rbind(hf_negative)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
print("positive instances: ",hf_positive.shape)
print("negative instances: ",hf_negative.shape)
print("total instances: ",hf.shape)

positive instances:  (165179, 34)
negative instances:  (283367, 34)
total instances:  (448546, 34)


In [7]:
hf = hf[['vgg_cosine', 'vgg_euclidean_l2'
         , 'facenet_cosine', 'facenet_euclidean_l2'
         , 'openface_cosine', 'openface_euclidean_l2'
         , 'is_related']]

In [8]:
hf.head()

vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
0.618396,1.11211,1.25131,1.58197,1.12544,1.50029,1
0.601191,1.09653,1.14205,1.51133,1.08315,1.47183,1
0.543063,1.04217,1.10449,1.48627,1.14981,1.51645,1
0.618544,1.11224,1.24833,1.58008,1.09367,1.47897,1
0.60665,1.1015,1.15115,1.51733,1.11618,1.49411,1
0.630702,1.12312,1.22153,1.56303,1.20384,1.55167,1
0.742856,1.2189,1.09549,1.48019,1.07032,1.4631,1
0.558462,1.05685,1.11984,1.49655,1.24634,1.57882,1
0.648851,1.13917,1.10773,1.48844,0.612283,1.1066,1
0.677681,1.1642,1.03272,1.43716,0.571711,1.06931,1


In [10]:
#convert target label to factor because this is a binary classification
#otherwise, there would be a regression problem
hf['is_related'] = hf['is_related'].asfactor()

In [11]:
#70% train, 15% test, 15% validation
train, validation = hf.split_frame(ratios=[0.80], seed=17)

# Training

In [12]:
from h2o.automl import H2OAutoML

In [13]:
y_label = hf.columns[-1]
x_labels = hf.columns[0:-1]

In [14]:
x_labels

['vgg_cosine',
 'vgg_euclidean_l2',
 'facenet_cosine',
 'facenet_euclidean_l2',
 'openface_cosine',
 'openface_euclidean_l2']

In [15]:
y_label

'is_related'

In [16]:
#this is a binary classification problem. convert is_related column to enum type instead of numerical
#otherwise, it would be a regression problem
hf[y_label] = hf[y_label].asfactor()

In [17]:
hf.describe()

Rows:448546
Cols:7




,vgg_cosine,vgg_euclidean_l2,facenet_cosine,facenet_euclidean_l2,openface_cosine,openface_euclidean_l2,is_related
type,real,real,real,real,real,real,enum
mins,0.004028141498565674,0.08975721,0.009035706520080566,0.1344302,0.002826511859893799,0.075187616,
mean,0.5712317096626222,1.0612570454616248,0.8638393964523046,1.3052493658216322,0.6978711954794291,1.1585682141629288,
maxs,1.0553148686885834,1.4528006,1.510623574256897,1.7381736,1.5858558416366575,1.78093,
sigma,0.13110342992075416,0.12726718868896808,0.19023790294772652,0.15492882258680396,0.2620664572912607,0.231219167951997,
zeros,0,0,0,0,0,0,
missing,0,0,0,0,0,0,0
0,0.6183964312076569,1.1121118,1.2513097524642944,1.581967,1.1254418790340424,1.5002946,1
1,0.6011905074119568,1.0965314,1.1420531868934631,1.5113261,1.083145149052143,1.4718323,1
2,0.5430631935596466,1.0421739,1.104493826627731,1.4862664,1.1498085260391235,1.5164489,1


In [18]:
aml = H2OAutoML(max_runtime_secs=60*60*2)

In [19]:
aml.train(x = x_labels, y = y_label, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [20]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20190918_120315

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.18602871080945219
RMSE: 0.4313104575702428
LogLoss: 0.5519274605556048
Null degrees of freedom: 358983
Residual degrees of freedom: 358970
Null deviance: 472505.93965882086
Residual deviance: 396266.25500018644
AIC: 396294.25500018644
AUC: 0.7683598034537101
pr_auc: 0.6631557027238266
Gini: 0.5367196069074203

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3266297610408758: 

,,0,1,Error,Rate
0,0,147909.0,78814.0,0.3476,(78814.0/226723.0)
1,1,35017.0,97244.0,0.2648,(35017.0/132261.0)
2,Total,182926.0,176058.0,0.3171,(113831.0/358984.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.326630,0.630801,251.0
1,max f2,0.190096,0.776442,346.0
2,max f0point5,0.492011,0.615327,159.0
3,max accuracy,0.492011,0.718422,159.0
4,max precision,0.884706,1.000000,0.0
5,max recall,0.131791,1.000000,386.0
6,max specificity,0.884706,1.000000,0.0
7,max absolute_mcc,0.376293,0.377670,223.0
8,max min_per_class_accuracy,0.351284,0.690746,236.0
9,max mean_per_class_accuracy,0.326630,0.693810,251.0



Gains/Lift Table: Avg response rate: 36.84 %, avg score: 36.61 %


...

,,0,1,Error,Rate
0,0,121997.0,104726.0,0.4619,(104726.0/226723.0)
1,1,30017.0,102244.0,0.227,(30017.0/132261.0)
2,Total,152014.0,206970.0,0.3753,(134743.0/358984.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.282476,0.602799,281.0
1,max f2,0.163269,0.752534,366.0
2,max f0point5,0.477865,0.576995,167.0
3,max accuracy,0.493934,0.695922,159.0
4,max precision,0.991759,1.000000,0.0
5,max recall,0.107488,1.000000,398.0
6,max specificity,0.991759,1.000000,0.0
7,max absolute_mcc,0.373526,0.322216,227.0
8,max min_per_class_accuracy,0.348367,0.664075,240.0
9,max mean_per_class_accuracy,0.358466,0.665518,235.0



Gains/Lift Table: Avg response rate: 36.84 %, avg score: 36.84 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010000,0.824756,2.377012,2.377012,0.875766,0.851066,0.875766,0.851066,0.023771,0.023771,137.701203,137.701203
1,,2,0.020001,0.794654,2.188000,2.282506,0.806128,0.808045,0.840947,0.829556,0.021881,0.045652,118.800026,128.250614
2,,3,0.030001,0.776259,2.131297,2.232103,0.785237,0.784847,0.822377,0.814653,0.021314,0.066966,113.129673,123.210300
3,,4,0.040002,0.758201,2.060228,2.189134,0.759053,0.767828,0.806546,0.802946,0.020603,0.087569,106.022830,118.913433
4,,5,0.050002,0.738468,1.997476,2.150803,0.735933,0.748222,0.792423,0.792002,0.019976,0.107545,99.747639,115.080274
5,,6,0.100002,0.659521,1.837297,1.994054,0.676918,0.696796,0.734672,0.744400,0.091864,0.199409,83.729677,99.405412
6,,7,0.150001,0.594231,1.663094,1.883736,0.612736,0.625593,0.694028,0.704798,0.083154,0.282563,66.309381,88.373607
7,,8,0.200001,0.536234,1.490705,1.785480,0.549223,0.564806,0.657827,0.669801,0.074534,0.357097,49.070548,78.547979
8,,9,0.299999,0.451846,1.336917,1.635960,0.492562,0.493216,0.602739,0.610940,0.133690,0.490787,33.691693,63.596022
9,,10,0.400001,0.384709,1.154592,1.515616,0.425388,0.417618,0.558401,0.562609,0.115461,0.606248,15.459166,51.561641


# Leaderboard

In [22]:
lb = aml.leaderboard

In [23]:
#lb.head()
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190918_120315,0.725348,0.581563,0.344432,0.445367,0.198352
StackedEnsemble_BestOfFamily_AutoML_20190918_120315,0.725319,0.581576,0.343276,0.44537,0.198354
XGBoost_grid_1_AutoML_20190918_120315_model_1,0.725097,0.580942,0.343419,0.445203,0.198206
XGBoost_3_AutoML_20190918_120315,0.725071,0.580922,0.343737,0.445195,0.198198
GBM_1_AutoML_20190918_120315,0.724961,0.581041,0.342287,0.44525,0.198248
GBM_grid_1_AutoML_20190918_120315_model_1,0.7249,0.5813,0.344243,0.445321,0.198311
GBM_2_AutoML_20190918_120315,0.724727,0.581126,0.343028,0.445301,0.198293
GBM_grid_1_AutoML_20190918_120315_model_7,0.724651,0.58152,0.343628,0.445424,0.198403
GBM_3_AutoML_20190918_120315,0.72437,0.581478,0.344385,0.445462,0.198437
GBM_grid_1_AutoML_20190918_120315_model_4,0.723987,0.585939,0.34366,0.4472,0.199988


In [24]:
saved_model = h2o.save_model(aml.leader, path = "", force=True)

In [25]:
saved_model

'/outputs/sefik/kinship/StackedEnsemble_AllModels_AutoML_20190918_120315'

# Load the best model

In [26]:
#you can directly load the best model by running the load_model command
restored_aml = h2o.load_model(saved_model)

# Validation Score

In [29]:
perf = aml.leader.model_performance(validation)
#perf = restored_aml.model_performance(validation)

In [30]:
perf.auc()

0.724550507096832

In [31]:
perf.accuracy() [0][1]

0.6957303320604721